# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
#import gmaps
import geopandas as gpd
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,22.53,90,89,3.58,EC,1726337867
1,1,colonia,50.9333,6.9500,14.36,73,0,2.06,DE,1726337843
2,2,port elizabeth,-33.9180,25.5701,16.84,94,20,2.06,ZA,1726337851
3,3,tilisarao,-32.7329,-65.2911,21.81,18,0,2.11,AR,1726337811
4,4,wailua homesteads,22.0669,-159.3780,27.44,78,40,6.69,US,1726337873


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
humidity_map



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#filtering for temp between 21 and 27C, Wind Speed less than 4.5 m/s, Cloudiness=0
city_ideal = city_data_df.loc[(city_data_df["Max Temp"]>=21) & (city_data_df["Max Temp"]<=27) & (city_data_df["Wind Speed"]<=4.5) & (city_data_df["Cloudiness"]<=0)]

# Drop any rows with null values
city_clean = city_ideal.dropna()

# Display sample data
city_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,tilisarao,-32.7329,-65.2911,21.81,18,0,2.11,AR,1726337811
84,84,kathua,32.3667,75.5167,24.07,83,0,2.23,IN,1726337971
120,120,miles city,46.4083,-105.8406,22.54,37,0,2.06,US,1726338012
225,225,chibougamau,49.9168,-74.3659,23.79,43,0,2.06,CA,1726338136
233,233,beira,-19.8436,34.8389,23.05,94,0,3.09,MZ,1726338146
262,262,zahedan,29.4963,60.8629,23.95,19,0,3.09,IR,1726338180
270,270,la ligua,-32.4524,-71.2311,23.16,41,0,3.99,CL,1726338189
305,305,temiskaming shores,47.4938,-79.7153,26.89,41,0,1.82,CA,1726338229
324,324,arroyo,41.6096,-4.7969,24.02,20,0,2.06,ES,1726338252
325,325,fasa,28.9383,53.6482,25.58,25,0,1.03,IR,1726338253


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_clean[["Lat", "Lng", "City", "Country"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,Lat,Lng,City,Country,Hotel Name
3,-32.7329,-65.2911,tilisarao,AR,
84,32.3667,75.5167,kathua,IN,
120,46.4083,-105.8406,miles city,US,
225,49.9168,-74.3659,chibougamau,CA,
233,-19.8436,34.8389,beira,MZ,
262,29.4963,60.8629,zahedan,IR,
270,-32.4524,-71.2311,la ligua,CL,
305,47.4938,-79.7153,temiskaming shores,CA,
324,41.6096,-4.7969,arroyo,ES,
325,28.9383,53.6482,fasa,IR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
   "limit":limit,
   "categories": f"accommodation.hotel",
   "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tilisarao - nearest hotel: No hotel found
kathua - nearest hotel: No hotel found
miles city - nearest hotel: Historic Olive Hotel
chibougamau - nearest hotel: No hotel found
beira - nearest hotel: Hotel Savoy
zahedan - nearest hotel: هتل صالح;بانک ملی
la ligua - nearest hotel: Hotel Anchimallen
temiskaming shores - nearest hotel: Waterfront Inn
arroyo - nearest hotel: El Jardín de la Abadía
fasa - nearest hotel: هتل جهانگردی فسا
sidi salim - nearest hotel: No hotel found
presque isle - nearest hotel: The Northeastland Hotel
jackson - nearest hotel: Brookside


,Lat,Lng,City,Country,Hotel Name
3,-32.7329,-65.2911,tilisarao,AR,No hotel found
84,32.3667,75.5167,kathua,IN,No hotel found
120,46.4083,-105.8406,miles city,US,Historic Olive Hotel
225,49.9168,-74.3659,chibougamau,CA,No hotel found
233,-19.8436,34.8389,beira,MZ,Hotel Savoy
262,29.4963,60.8629,zahedan,IR,هتل صالح;بانک ملی
270,-32.4524,-71.2311,la ligua,CL,Hotel Anchimallen
305,47.4938,-79.7153,temiskaming shores,CA,Waterfront Inn
324,41.6096,-4.7969,arroyo,ES,El Jardín de la Abadía
325,28.9383,53.6482,fasa,IR,هتل جهانگردی فسا


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)